# Interaction Inference

This notebook walks through the main functionality of the package and provides some background.

## Background

Gene expression is a process that occurs within cells where sections of DNA are 'transcribed' into copies called transcripts that carry information, such as how to produce proteins, to the cell machinery where they are 'translated' into functional products (Or a wide variety of other processes, this is a very simplified explanation [see more here.](https://www.genome.gov/genetics-glossary/Gene-Expression))

However, genes do not act independently and often interact with each other to regulate transcription, forming large and complex 'gene regulatory networks'. The package focuses on the problem of detecting if two genes are interacting given observations of their transcriptional behaviour.

We focus on a simple 'birth-death interaction' model of transcription in the gene pair, a stochastic reaction network (markov chain) where transcripts from each gene, $X_{1}$ and $X_{2}$, are produced (birth), degraded (death) and can regulate each other:

$$ \varnothing \stackrel{k_{tx, 1}}\longrightarrow X_{1} \qquad \varnothing \stackrel{k_{tx, 2}}\longrightarrow X_{2} $$
$$ X_{1} \stackrel{k_{deg, 1}}\longrightarrow \varnothing \qquad X_{2} \stackrel{k_{deg, 2}}\longrightarrow \varnothing $$
$$ X_{1} + X_{2} \stackrel{k_{reg}}\longrightarrow \varnothing $$

Detecting interaction then reduces to testing if a model with interaction strength $k_{reg} > 0$ is consistent with observed data.

## Package overview

This package implements two optimization based methods and a baseline correlation test for inference of interaction from datasets of transcripts. It also contains functions to simulate example datasets from the above 'birth-death interaction' model and to compute + display the classification performance of the methods using the true model parameters.

The optimization methods use the bootstrap to estimate bounds on the distribution of observed transcripts, combining constraints from the model structure to produce an optimization problem over the model parameters. The 'Minimization' approach optimizes a lower bound on the interaction parameter, and if this is found to be > 0 can conclude evidence of interaction. The 'Hypothesis' approach assumes no interaction is present, fixing the parameter = 0, and tests the feasibility of the constraints under this assumption.

# Tutorial

The package centers around a dataset class and classes for inference methods which can each be used with minimal setup, or fine tuned for more control

In [ ]:
# import the dataset class
from interaction_inference import Dataset

# import inference method classes
from interaction_inference import Hypothesis, Minimization, Correlation

# import result analysis module
from interaction_inference import results

## Dataset

Create an instance of the dataset class and simulate an example dataset

In [ ]:
data = Dataset()

data.simulate_dataset(
    beta=0.9,
    gene_pairs=50,
    cells=1000
)

The parameter beta is the 'capture efficiency' of the observation method, a value of 0.9 means that each transcript in the cell has a 90% chance to be 'captured' by the single-cell-sequencing method used to measure transcript levels. A lower capture efficiency means less information is observed and it is harder to detect interaction, with values often as low as 6%.

The pandas dataframes containing transcript values and model parameters for each gene pair can be saved and loaded as csv files:

In [ ]:
data.store_dataset(
    count_dataset_filename="example_transcript_dataset.csv",
    param_dataset_filename="example_parameter_dataset.csv"
)

In [ ]:
data.load_dataset(
    count_dataset_filename="example_transcript_dataset.csv",
    param_dataset_filename="example_parameter_dataset.csv",
    beta = 0.9
)

Display a scatter plot of the simulated parameters

In [ ]:
results.scatter_parameters(data)

## Inference

To apply inference methods to a dataset create an instance of its class and pass the dataset object to the analysis method. The default settings for each method should perform well in most cases, but can easily be changed by modifying attributes of the instance.

In [ ]:
# instance of hypothesis class
hyp = Hypothesis()

# analyse dataset
hyp.analyse_dataset(data)

In [ ]:
# instance of minimization class
minz = Minimization()

# analyse dataset
minz.analyse_dataset(data)

In [ ]:
# instance of correlation class
corr = Correlation()

# analyse dataset
corr.analyse_dataset(data)

## Result analysis

The results of analysis are stored as a dictionary in the .result_dict attribute of the method object.

If the true parameters are available (i.e. simulated dataset) the package provides functions to display a scatter plot of the classification into interating vs non-interacting gene pairs, and functions to compute classification performance metrics:

### Scatter plots

In [ ]:
results.scatter_results(hyp)

In [ ]:
results.scatter_results(minz)

In [ ]:
results.scatter_results(corr)

### Performance metrics

In [ ]:
results.classification_performance(hyp)

In [ ]:
results.classification_performance(minz)

In [ ]:
results.classification_performance(corr)